In [1]:
from langchain_chroma import Chroma


In [2]:
# create embedding model object
import torch

In [3]:
if torch.backends.mps.is_available():
    mps_device = torch.device('mps')
    x = torch.ones(5, device=mps_device)
    print(x)
else:
    print("device mps not found")

tensor([1., 1., 1., 1., 1.], device='mps:0')


In [4]:
# call indobert transformers
from transformers import BertTokenizer, AutoModel

tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

/Users/a/Programming/Langchain-Project/my-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7ab62a42-bc68-44ec-b58a-11e70dcf5178)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/indobenchmark/indobert-base-p1/c2cd0b51ddce6580eb35263b39b0a1e5fb0a39e2/config.json
Retrying in 1s [Retry 1/5].


KeyboardInterrupt: 

In [5]:
x = torch.LongTensor(tokenizer.encode('nama saya adalah romi')).view(1,-1)
# print(x, model(x)[0].sum())
print(x.shape)

torch.Size([1, 7])


In [6]:
# test embedding model dengan indobert

x = torch.LongTensor(tokenizer.encode("romi")).view(1, -1)

In [ ]:
print(x, model(x)[0].shape, model(x)[1].shape)

tensor([[    2,  3610, 30356,     3]]) torch.Size([1, 4, 768]) torch.Size([1, 768])


In [8]:
# save data vector with chroma on memory not persistance and this will be not save on disk

vector_store = Chroma(
    collection_name="test-collection",
    embedding_function=tokenizer,
)

In [7]:
# untuk berinteraksi dengan chromadb, kita bisa menggunkana ChromaClient (ini adalah chroma client in memory bukan persistance)

import chromadb

client = chromadb.Client()

In [8]:
# setelah client berhasil dibuat, kemudian connect to server

client = chromadb.HttpClient()

### Ternyata untuk menggunakan chromadb, kita wajib menginstall chromadb diluar langchain, dan setiap ingin menjalankan harus menggunakan perintah chroma run (chroma akan jalan pada port 8000 secara default)

### Jadi setelah chromadb client berhasil dibuat dan telah berhasil connect ke chromadb server, selanjutnya adalah membuat collection sebagai tempat untuk menyimpan vector atau vector store, document, also any additional data

In [9]:
collection = client.get_or_create_collection(name="test-collection")

In [39]:
collection_list = client.list_collections()

In [40]:
for instance in collection_list:
    print(instance)

Collection(name=test-collection)


In [21]:
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

/Users/a/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:10<00:00, 1.17MiB/s]


In [ ]:
# menambahkan document kedalam collection (test-collection dengan menggunakan vector store yang telah kita buat dengan lanchain)


In [ ]:
from langchain_core.documents import Document

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

In [ ]:
# this code is test rag-branch


### saya punya hipotesis, sepertinya chromadb dengan lanchain tidak perlu mendowload lagi chromadb, cari di sini saya akan membuat object chroma baru untuk data persistence

from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

### oke, ternyata untuk embedding model, karena saya menggunakan model indobert yang mana tokenizer (proses mengubah teks menjadi angka) dan embedding model-nya terpisah, maka saya akan membuat satu buah function yang menjadi pipe teks masuk.

### ternyata tidak bisa hanya menggunakan pure function saja, kita harus membuat sebuah class yang terhubung dengan embedding langchain

### dan lagi, ternyata inference model embedding seperti ini sudah tersedia melalui integrasi langchain dengan huggingface

In [9]:
# di sini, karen sudah mendownload transformer, maka kita gunakan class python dengan menghubungkannya dengan lanchain
from typing import List
from langchain_core.embeddings import Embeddings

class IndoBERTEmbeddings(Embeddings):
    def __init__(self, model_name="indobenchmark/indobert-base-p1"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval() # supaya model tidak evaluasi (?)


    def _generate_embedding(self, text: str) -> List[float]:
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

        # masuk ke model dengan torch
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # polling token menjadi satu vector kalimat 
        token_embeddings = outputs.last_hidden_state

        # melakukan mean polling 
        sentence_embeddings = token_embeddings.mean(dim=1)

        # konversi ke list python
        return sentence_embeddings.squeeze().tolist()
    

    # menyimpan document pada chroma
    def embed_documents(self, texts: List[str]) -> List[List[float]] :
        return [self._generate_embedding(text) for text in texts]
    

    # metode untuk pencarian query pada chroma
    def embed_query(self, text:str) -> List[float] :
        return self._generate_embedding(text)
        

In [10]:
# membuat objek class dari model IndoBERTEmbeddings

embeddings = IndoBERTEmbeddings()

In [11]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="chroma-with-langchain",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [ ]:
# mari kita masukkan document kedalam collection chroma-with-lanchain

from langchain_core.documents import Document

data_teks = [
    Document(
        page_content="Presiden pertama Indonesia adalah Ir. Soekarno.",
        metadata={"sumber": "buku_sejarah", "tahun": 1945}
    ),
    Document(
        page_content="Rendang adalah makanan khas dari Sumatera Barat yang sangat lezat.",
        metadata={"sumber": "kuliner", "rasa": "pedas"}
    ),
    Document(
        page_content="IndoBERT adalah model bahasa yang dilatih khusus untuk bahasa Indonesia.",
        metadata={"sumber": "tech_article", "kategori": "AI"}
    ),
    Document(
        page_content="Candi Borobudur terletak di Magelang, Jawa Tengah.",
        metadata={"sumber": "wisata", "tipe": "candi"}
    ),
]


In [18]:
vector_store.add_documents(documents=data_teks)

['e8e234dd-5c56-4b75-a2ea-bc03eebce80a',
 'a28cc805-0206-4209-ad5b-80a02c1bc400',
 '5e7d1c89-d2e5-4986-853a-f4db9aae5b8e',
 '79c87949-4b71-4b1c-b2b2-5219050524d1']

### Dokumen-dokumen telah berhasil dimasukkan ke dalam collection, kita coba retrieve dokumen berbdasarkan query

In [19]:
query = "siapa pemimpin pertama Indonesia"

In [20]:
result = vector_store.similarity_search(query, k=1)

In [23]:
for doc in result:
    print(doc.page_content)
    print(doc.metadata)

Presiden pertama Indonesia adalah Ir. Soekarno.
{'tahun': 1945, 'sumber': 'buku_sejarah'}
